## **Sum To N**

**Problem:**

Let's say we have some `array` with `r` numbers $\in \{1..50\}$

And we want to find the subset of `array`, such that the sum of its elements are closest to `N` without going over.

_Note:_
- an ideal solution is one where the chosen subset sums to exactly `N`

- an ideal solution may not exist

- multiple 'best' solutions may exist


In [1]:
import random
random.seed(12) # for reproducability

N = 0
array = []

def init_problem(size):
	global array, N
	array = [random.randint(1,50) for _ in range(size)]
	N = random.randint(0, 50 * size)

init_problem(10)
print(f'array: {array}')

array: [31, 18, 43, 34, 43, 23, 10, 25, 1, 24]


We have an array of size `10` containing random numbers between 1 and 50.

- The **largest** possible sum is 50,  
when the entire array is selected and all of its elements are 50.

- The **smallest** possible sum is 0, when no elements are selected.

We will try to reach our random target `N`.

In [2]:
print(f'N: {N}')

N: 247


## **Brute Force Approach**

To solve this problem using **brute force**, we would need to iterate through every possible subset of the array,  
tracking the closest sum to `N` we find until we reach `N` or exhaust all possible subsets.

We will represent a subset as a binary string such that the $i^{th}$ bit determines whether the $i^{th}$ element in the array exists in the set.  
A `1` indicates the element is present; `0` indicates the element is missing.

For instance, we have some array in a fixed order: `[a, b, c, d]`

The bitstring `0110` represents the subset: `[b, c]`

In [3]:
def bf_sum_to_n(arr, target):
	closest_sum = 0
	closest_subset = [] # empty set (default solution)
	
	arr_len = len(arr) # there will be 2^(arr_len) possible subsets
	for subset_mask in range(1 << arr_len): # all binary numbers from 1 to 2^(arr_len)
		subset = []
		for j in range(arr_len):
			if subset_mask & (1 << j) > 0: # the jth bit of the mask is set (>0)
				''' Ex.
					subset_mask 			= 0001 0000
					(1 << j) 				= 0000 0001
					subset_mask & (1 << j) 	= 0000 0000
					=> The jth bit of the mask was unset (0)
				'''
				subset.append(arr[j])
		# check for a new closest
		subset_sum = sum(subset)
		if subset_sum == target:
			return subset # found a solution
		elif subset_sum > closest_sum and subset_sum < target:
			closest_sum = subset_sum
			closest_subset = subset
	return closest_subset

In [4]:
ans = bf_sum_to_n(array, N)
print(f'array: {array}\nN: {N}\n')
print(f'solution: {ans}\nsum: {sum(ans)}')

array: [31, 18, 43, 34, 43, 23, 10, 25, 1, 24]
N: 247

solution: [31, 18, 43, 34, 43, 23, 25, 1, 24]
sum: 242


Let us check the time it takes to find the optimal solution:

In [5]:
%time ans = bf_sum_to_n(array, N)

CPU times: total: 0 ns
Wall time: 2.99 ms


Although this seems pretty fast for a small array,  
the **brute force** solution could be very computationally expensive.  

There are a total of $2^{r}$ subsets of our initial array.  
The problem is that the number of subsets scales exponentially with the size of array, `r`.

In our case, $2^{10} = 1024$,  
but if `r = 30`, for instance, we would potentially need to check $2^{30} = 1.07$ billion subsets.

Based on our wall time, it took $\approx 3 \, \text{ms}$ to check 1024 subsets.  
Or, roughly $2.92 \, \mu\text{s}$ per subset.

$\therefore$ for 1.07 billion subsets, we can imagine the execution time would take
$\approx 52 \, \text{minutes}$


In [6]:
# wall time per subset
ms_per_subset = 3/1024 # ms
print(f'{ms_per_subset} ms per subset')
# estimate time for checking 1.07 billion subsets
# convert time to minutes
t = ms_per_subset * 1_070_000_000 / 1000 / 60 # mins
print(f'{t:.2f} mins for 1.07 billion subsets')

0.0029296875 ms per subset
52.25 mins for 1.07 billion subsets


Now, let's try a new problem where array size `r = 30`:

In [7]:
# create new array of size 30, and a new target N
# we will choose a specific problem for reproducability
array = [41, 40, 12, 7, 29, 20, 10, 6, 35, 45, 41, 3, 39, 26, 29, 42, 48, 40, 42, 11, 40, 1, 34, 5, 4, 3, 13, 16, 39, 2]
N = 950

print(f'array: {array}')
print(f'N: {N}\n')

array: [41, 40, 12, 7, 29, 20, 10, 6, 35, 45, 41, 3, 39, 26, 29, 42, 48, 40, 42, 11, 40, 1, 34, 5, 4, 3, 13, 16, 39, 2]
N: 950



#### We don't have 52 minutes to waste! We need a better solution!

In [8]:
# This will take too long to run. Brute force is ineffective.
# ans = bf_sum_to_n(array, N)

## **Use a Genetic Algorithm**

- each subset is a potential solution

- all our solutions, therefore, are already in a coded format:&ensp; `a binary string`

We need to define a `fitness function` to evaluate how close our subset is to the optimal solution.

We can use the binary string to count a sum from the array, and compare its absolute value to `N`.

- binary strings that are closer to N will be preserved to the next generation

- binary strings that are _weak_ will be killed off

Now, we just need a way to make _baby_ solutions.

We can choose two parents (binary strings), split them at a random point, and rejoin the ends to  
__cross over__ the coded information.

We can also introduce a rare chance to __mutate__ a gene, by flipping a bit at a random position.


In [9]:
class Individual:
	''' An individual representing a subset of the array

	Attributes:
		length (int): Size of binary string (array)
		dna (str): Encoded binary string representing a particular subset
		fitness (int): How healthy this solution is (lower is better)
	'''
	MUTATION_RATE = 0.10

	def __init__(self, length, dna=None):
		self.length = length
		if dna is None:
			self.randomize_genes() # sets self.dna
		else:
			self.dna = dna
		self.evaluate_fitness() # sets self.fitness

	def randomize_genes(self):
		self.dna = ''.join(random.choices('01', k=self.length))

	def dna_to_subset(self):
		''' Cast the binary dna sequence to a sublist of the array '''
		subset = []
		for i in range(self.length):
			if self.dna[i] == '1':
				subset.append(array[i])
		return subset

	def evaluate_fitness(self):
		''' Compute subset sum, and return its distance from N, if less than or equal to N.

			Otherwise, incur a severe penalty for going over N.
		'''
		subset = self.dna_to_subset()
		self.fitness = N - sum(subset)
		if self.fitness < 0: # subset sum is larger than N
			self.fitness = N + 1 # worse than the empty subset (N - 0)

	def cross_over(self, other):
		''' Slice the parents' dna at a random point,  
		and recombine the strands to form 2 children DNA strings. '''
		i = random.randint(1, self.length - 1)
		strand_1 = self.dna[:i] + other.dna[i:]
		strand_2 = other.dna[:i] + self.dna[i:]
		return (strand_1, strand_2)

	@staticmethod
	def mutate(dna):
		''' Random chance to mutate each gene (bit) in the dna seqeunce. '''
		new_dna = []
		n = len(dna)
		for i in range(n):
			r = random.random()
			if r < Individual.MUTATION_RATE:
				new_dna.append('1') if dna[i] == '0' else new_dna.append('0')
			else:
				new_dna.append(dna[i])
		return ''.join(new_dna)

	def procreate(self, other):
		''' Create 2 children by crossing over the parents' dna and obtaining 2 new strands.

		- Add a chance to randomly mutate the childrens' dna at random points.
		'''
		if self.length != other.length:
			raise ValueError('individuals have dna of different lengths')
		strands = self.cross_over(other)
		zygotes = [Individual(self.length, Individual.mutate(z)) for z in strands]
		return zygotes
	
	def __str__(self):
		return self.dna


Let's create an initial population of `8` members with randomly generated genes.

- Each generation, we will pick the __top 3__ fittest individuals and pair them up in 3 different ways, each producing 2 offspring.

- We will have __6 new solutions__ born into the population.

- Then, we will kill off the __weakest 6__ members to mainting a population size of 8.  
	Note that this may include killing off the children—if they were born with worse genetics than the others.

Each population after reprduction represents another generation.

We will continue this process for just `10` generations, or until the ideal solution was found (the _golden child_)

In [10]:
max_generations = 10

def init_population(pop_size):
	dna_length = len(array)
	pop = [Individual(dna_length) for _ in range(pop_size)]
	pop.sort(key=lambda x : x.fitness) # ascending order based on an individual's fitness
	return pop

# initial society of 8
population_state = random.getstate() # we can restore to this point in the RNG later
population = init_population(8)
generation = 0

print('individual # | dna sequence | fitness score (lower is healthier)\n')

def show_population(pop):
	for i, member in enumerate(pop):
		print(f'\tmember {i+1})  {member}  {member.fitness}')

show_population(population)

individual # | dna sequence | fitness score (lower is healthier)

	member 1)  010001111110111100110111111100  488
	member 2)  100000011100110011011011111010  521
	member 3)  011110011101010110010111100011  561
	member 4)  000001001011011001111010000010  590
	member 5)  011111000101010010011011000010  591
	member 6)  100100100011110011000111010100  636
	member 7)  001111011000010001111000001000  669
	member 8)  010101000100000101010011001000  693


In [11]:
def evolve(population, max_generations, starting_gen=0, show_output=True):
	''' Evlolve the population for another `max_generations`

		starting_gen (int): the generation the population is starting from
	'''
	n = 0
	while n < max_generations:
		# check for 'golden child'
		if population[0].dna == 0: # perfect genetics (ideal solution)			
			break
		# selection
		strongest = population[:3] # top 3 fit
		children = []
		for i in range(3):
			for j in range(i+1, 3): # mate with others in the top 3
				children.extend(strongest[i].procreate(strongest[j]))
		population += children # add to the society
		# replacement
		population.sort(key=lambda x : x.fitness)
		del population[8:] # only the strongest survive
		# display next generation
		n += 1
		if show_output:
			print(f'Generation {starting_gen + n}')
			show_population(population)
			print()
	return n # the number of generations passed


Ok, let's look back at our problem:

In [12]:
print(f'array: {array}')
print(f'N: {N}\n')

# max subset sum (including all elements)
print(f'sum(array) = {sum(array)}')
print(f'N - sum(array) = {N - sum(array)}')

array: [41, 40, 12, 7, 29, 20, 10, 6, 35, 45, 41, 3, 39, 26, 29, 42, 48, 40, 42, 11, 40, 1, 34, 5, 4, 3, 13, 16, 39, 2]
N: 950

sum(array) = 723
N - sum(array) = 227


So, the target which we are attempting to sum to (but not exceed) is `N = 950`.

However, the sum of the entire array is only `723`.  
- This means that the best solution is the subset equal to the original array.

- It's not an ideal solution, but it is the best possible.

- Additionally, the fitness of the best solution is `227`.  
This is what we want members of our population to become.

Here is our random starting population one more time, for reference:

In [13]:
print('Generation 0')
show_population(population)

Generation 0
	member 1)  010001111110111100110111111100  488
	member 2)  100000011100110011011011111010  521
	member 3)  011110011101010110010111100011  561
	member 4)  000001001011011001111010000010  590
	member 5)  011111000101010010011011000010  591
	member 6)  100100100011110011000111010100  636
	member 7)  001111011000010001111000001000  669
	member 8)  010101000100000101010011001000  693


In [25]:
# re-run this cell to obtain a healthier population,
# and watch the gene pool (solutions) converge towards optimality.
generation += evolve(population, max_generations, generation)

Generation 111
	member 1)  111111111111111111111111111110  229
	member 2)  111111111111111111111111111110  229
	member 3)  111111111111111111111111111110  229
	member 4)  111111111111111111111111111110  229
	member 5)  111111111111111111111111111110  229
	member 6)  111111111111111111111111111110  229
	member 7)  111111111111111111111111111110  229
	member 8)  111111111111111111111111111110  229

Generation 112
	member 1)  111111111111111111111111111110  229
	member 2)  111111111111111111111111111110  229
	member 3)  111111111111111111111111111110  229
	member 4)  111111111111111111111111111110  229
	member 5)  111111111111111111111111111110  229
	member 6)  111111111111111111111111111110  229
	member 7)  111111111111111111111111111110  229
	member 8)  111111111111111111111111111110  229

Generation 113
	member 1)  111111111111111111111111111110  229
	member 2)  111111111111111111111111111110  229
	member 3)  111111111111111111111111111110  229
	member 4)  11111111111111111111111111111

Wow, after about $120$ generations, we were able to obtain the best possible solution!

Note that it won't always take the same number of generations to achieve the optimal answer,  
as our genetic algorithm relies on random events (like mutation and crossover).

__Observations:__

- We noticed that the population started off becoming considerably more fit each generation.

- As more generations passed, only small improvements were made,  
and it took longer for the population to converge to the healthiest individual.

- This is because we are relying on more precise events to occur to either:
	1. Pass on the right genes to the children
	
	2. Or, wait for random mutation to hit the right gene

Next, we can make refinements to our genetic algorithm.

For instance, adjacent bits on a bit-string are not related to each other for the __Sum-to-N__ problem.  
This means that the inclusion of the $i^{\text{th}}$ element in the sum does not indicate that the $(i+1)^{\text{st}}$ element should or shouldn't be included.

Therefore, we can change the `crossover function` to employ __uniform crossover__.

In uniform crossover, each bit is chosen from either parent with equal probability.

This way, the differences in the parents' DNA are _blended_ together, while genes that are indentical are preserved.  
This will introduce more variation into the children.

In [26]:
def uniform_crossover(self, other):
	''' Blend dna together by randomly selecting a parent to take the ith gene from. 
	
	Return the 2 new binary strings.
	'''
	strands = []
	for _ in range(2):
		new_dna = []
		for i in range(self.length):
			if random.random() < 0.5:
				new_dna.append(self.dna[i])
			else:
				new_dna.append(other.dna[i])
		strands.append(''.join(new_dna))
	return strands

Individual.cross_over = uniform_crossover

In [27]:
# let's use the same initial population as before
random.setstate(population_state)
generation = 0
population = init_population(8)
show_population(population)

	member 1)  010001111110111100110111111100  488
	member 2)  100000011100110011011011111010  521
	member 3)  011110011101010110010111100011  561
	member 4)  000001001011011001111010000010  590
	member 5)  011111000101010010011011000010  591
	member 6)  100100100011110011000111010100  636
	member 7)  001111011000010001111000001000  669
	member 8)  010101000100000101010011001000  693


In [31]:
# re-run this cell to obtain a healthier population,
# and watch the gene pool (solutions) converge towards optimality.
generation += evolve(population, max_generations, generation)

Generation 31
	member 1)  111111111110111111111111111111  230
	member 2)  111111111111111111111110111111  232
	member 3)  111111111110111111111110111111  235
	member 4)  111111111110111111111011011111  235
	member 5)  111111111110111111111110111111  235
	member 6)  111111111111111111111110101110  237
	member 7)  111111111110111111111110101111  238
	member 8)  111111111110111111111010011111  240

Generation 32
	member 1)  111111111110111111111111111111  230
	member 2)  111111111111111111111110111111  232
	member 3)  111111111110111111111110111111  235
	member 4)  111111111110111111111011011111  235
	member 5)  111111111110111111111110111111  235
	member 6)  111111111111111111111110101110  237
	member 7)  111111111110111111111110101111  238
	member 8)  111111111110111111111010011111  240

Generation 33
	member 1)  111111111110111111111111111111  230
	member 2)  111111111111111111111110111111  232
	member 3)  111111111110111111111110111111  235
	member 4)  111111111110111111111011011111  

This time, it only took $40$ generations for our strongest member to represent the best solutionQ

It appears that our population now converges towards the same healthiest individual at a faster rate.

Now, let's try a new problem with a _very large_ array.

We will set N to the maximum possible array sum `50*len(array)`,  
so that we know the ideal answer is the subet equal to the entire array. 

Then, we can compare our solution to the binary string of all 1's, to estimate the accuracy of our solution.

For instance, we can count the number of 1's in our solution's binary string and divide that by the total length of the string.

> 1010 $\rightarrow \frac{2}{4}$ or $50\%$ strength

In [32]:
init_problem(100) # array of size 100
N = 50 * len(array) # 5,000
array_sum = sum(array)

print(f'N: {N}')
print(f'sum(array) = {array_sum}')
print(f'N - sum(array) = {N - array_sum}')
print(f'\nTherefore, the best solution has a fitness of {N - array_sum}.')

N: 5000
sum(array) = 2538
N - sum(array) = 2462

Therefore, the best solution has a fitness of 2462.


In [33]:
# random initial population
generation = 0
population = init_population(8)
show_population(population)

	member 1)  1110110111101110111111110011011010011100101100101111011111000000001111001101100010011100111001100111  3505
	member 2)  1000110100010010000000101110110010101101010011111110100001110110110010100111100110010110110010111110  3528
	member 3)  1010100001101001101110101111110100000101000100111111100000110011011100001111111011101101111111100001  3639
	member 4)  0110001011001100101101101100011010001100110001001110110111001111101001001111110001010110001110010010  3641
	member 5)  0111001100000111101011010010110000010111111110111101111000111000000101111010010100010011111001111001  3671
	member 6)  1001110001100100011000111111101001001001110001011011010100110111010011110011010010110011101010100011  3709
	member 7)  0011101011001001000011100111100101101100101010111111011101000100000010011100001011110101101011010100  3729
	member 8)  1010110001111011100100100110000001100100111001110100111110110110001110100101001010111101000000010010  3790


In [131]:
# let's see how many generations it takes to solve this complex problem
generation += evolve(population, max_generations, generation)

Generation 961
	member 1)  1111111111111111111110110111111111111010111111111111111111011111111111110111111111111110111110111111  2575
	member 2)  1111111011111111111110110111111111111010111111111110111111011111111111110111101111111110111111111110  2591
	member 3)  1011111111111111101110110111111111111010111111111111111111011111111111110111101111111110111111111110  2591
	member 4)  1111111111111111111110110111111111111011111111111110111111011111101111110111101111111110111110111110  2598
	member 5)  1111101001111111111110110111111111111010111111111111111111101111111111110111101111111111111111111110  2605
	member 6)  1111110110111011111100011111111111111111111111111110111111011110111111110111101111111110111111111110  2611
	member 7)  1011111111110111111110100111111111111011110111111111111111111111110101110111110111111110111111111111  2613
	member 8)  1111101011111111111100110111111111111010111111111110111111101111111111110111101111111110111111111110  2615

Generation 962
	member 1)  11111

In [132]:
def get_strength(dna: str): # ratio of 1's in a DNA bit-string
	return dna.count('1') / len(array)

# check the strength of our 'current best' solution
strength = get_strength(population[0].dna)
print(f'strength: {strength:.2%}')

strength: 92.00%


It's taking significantly more generations to find a good solution now.

Our algorithm seems to slow down exponentially once our solution achieves a strength of about $90\%$.

With such large bit-strings, there are so many possibilities for crossover and mutation now.

The result is slower convergence.

However, a $90\%$ accurate solution isn't a bad approximation,  
considering the brute force algorithm would be absolutely useless with this large array size.

In fact, let's estimate the time it would take for the brute force approach to compute the solution:

- We have $2^{100}$ subsets to search in this case

In [134]:
subsets = 2**100
t = ms_per_subset * subsets # milliseconds
t = t / 1000 / 60 / 60 / 24 / 365 # years
t /= 225_000_000 # galactic years
t /= 1_000_000 # millions of galactic years

print(f'It would take about {t:.2f} million, galactic years to brute force the solution for an array of size 100,')
print('where 1 galactic year is about 225 million years on Earth.')

It would take about 523.40 million, galactic years to brute force the solution for an array of size 100,
where 1 galactic year is about 225 million years on Earth.


The slight inaccuracy of our genetic algorithm doesn't seem like such a big problem now, does it?

—because at least we can _actually_ get a solution.

Now, let's continue the current __Sum-To-N__ problem with an array size of 100.

We'll run the genetic algorithm for 10,000 generations, track the time it takes to execute, and observe the accuracy of the result:

In [135]:
%%time
population = init_population(8) # new starting population
evolve(population, 10_000, starting_gen=0, show_output=False);

CPU times: total: 3.59 s
Wall time: 3.59 s


In [136]:
strongest = population[0]
strength = get_strength(strongest.dna)
ans = strongest.dna_to_subset()
sum_ans = sum(ans)

print(f'N: {N}')
print(f'sum(solution): {sum_ans}')
print(f'distance from N: {abs(N - sum_ans)}\n')

print(f'strongest member: {strongest}')
print(f'fitness: {strongest.fitness}')
print(f'strength: {strength:.2%}')

N: 5000
sum(solution): 2531
distance from N: 2469

strongest member: 0110111111111111111110111111111111111111111111111111111111111111111111111111111111111111111111111111
fitness: 2469
strength: 97.00%


After 10,000 generations, we have obtained a solution that is about __97%__ as effective compared to what's optimal.

And we have done this in about 4 seconds.

That's a really decent solution in no time at all!

At least its _much better_ than waiting around __523 million galactic years__ like our brute force solution would require.

Finally, let's test the accuracy of our estimated brute force execution times.  
We will focus on the __Sum-To-N__ problem with array size 30.

Before, we estimated a time of 52 minutes for the brute force solution to search all subsets.

We'll leave the `brute force` algorithm running in the background and observe how long it actually takes to run:

In [137]:
init_problem(30) # array of size 30
N = 50 * len(array) # so we can check the validity of the answer (which should be equal to the array)
bf_ans = []
%time bf_ans = bf_sum_to_n(array, N)

CPU times: total: 1h 27min 37s
Wall time: 1h 28min 16s


In [138]:
bf_ans_sum = sum(bf_ans)
array_sum = sum(array)
print(f'N: {N}')
print(f'sum(array): {array_sum}')
print(f'sum(solution): {bf_ans_sum}')
print(f'distance from N: {N - bf_ans_sum}\n')

N: 1500
sum(array): 705
sum(solution): 705
distance from N: 795



Next, we will expose some problems with simple genetic algorithms involving premature convergence.

### __Useful Links__

- [Avoid The Plague](./avoid_the_plague.ipynb)